# I. Import packages

In [145]:
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

# II Input Data and create utility functions

In [146]:
data_address = "data_small.xlsx"

def lst_to_dict(lst):
    mydict = dict()
    for sublist in lst:
        mydict[tuple(sublist[0:3])] = sublist[len(sublist) - 1]
    return mydict


def _2d_lst_to_dict(lst, V):
    mydict = dict()
    for v in V:
        for i in range(len(lst)):
            for j in range(len(lst)):
                mydict[(i, j, v)] = lst[i][j]
    return mydict

def p_arr_to_m_dict(arr, n, r):
    m_dict = {}
    for key1, key2, value, _ in arr:
        if (key1 <= n) and (key2 <= r):
            if (key1, key2) in m_dict:
                m_dict[(key1, key2)].append(value)
            else:
                m_dict[(key1, key2)] = [value]

    return m_dict


# III. Define Model

In [147]:

df = pd.read_excel(data_address, "set")

n = int(df.iloc[0, 2])
r = int(df.iloc[1, 2])
m = int(df.iloc[2, 2])
v = int(df.iloc[3, 2])
# set # index [1, n+1]
N_full = range(0,n+2)
N = range(1, n+1)
N_plus = range(1,n+2)
N_minus = range(0, n+1)
R = range(1, r+1)
R_minus = range(2, r+1)
M = range(1, m+1)
V = range(1, v+1)


# Parameter
# unit processing cost on machine m per unit of time
lamda = np.transpose(np.delete(np.array(pd.read_excel(
    data_address, "machine_cost")), 0, 1))[0]

# fixed cost of vehicle type v
F = np.transpose(np.delete(np.array(pd.read_excel(
    data_address, "vehicle_cost")), 0, 1))[0]

# variable cost of vehicle v per unit of time
Theta = np.transpose(np.delete(np.array(pd.read_excel(
    data_address, "vehicle_cost")), 0, 1))[1]

# weight of early delivery
mu = df.iloc[4, 2]
# weight of tardy delivery
fei = df.iloc[5, 2]

# process time of operation r of job j on machine m
# [job, operation, machine, time]
P = np.array(pd.read_excel(
    data_address, "processing_time"))

m_dict = p_arr_to_m_dict(P, n, r)
P = lst_to_dict(P)

print(m_dict)



# process machine matrix
# [job, operation, machine, value]
a = lst_to_dict(np.array(pd.read_excel(
    data_address, "process_machine_matrix")))


# transportation time from customer i to customer j by vehicle v
t = _2d_lst_to_dict(
    np.delete(np.array(pd.read_excel(
        data_address, "transport_time")), 0, 1), V)

# size of job j
theta = np.transpose(np.delete(np.array(pd.read_excel(
    data_address, "size_job")), 0, 1))[0]

# delivery time window of job j
tw = np.array(pd.read_excel(
    data_address, "delivery_time_window"))[:,1:]

# capacity of vehicle v
q = np.transpose(
    np.delete(np.array(pd.read_excel(
        data_address, "vehicle_capacity")), 0, 1))[0]

bigM = 1e10

# decision variable
# production

solver = pywraplp.Solver.CreateSolver("SCIP")

# production start time of operation r of job j
pi = {}
for j in N:
    for r in R:
        pi[(j, r)] = solver.NumVar(0, solver.infinity(), "")

# production completion time of operation r of job j
gamma = {}
for j in N_full:
    for r in R:
        gamma[(j, r)] = solver.NumVar(0, solver.infinity(), "")

# production complation time of job j
C = []
for j in N:
    C.append(solver.NumVar(0, solver.infinity(), ""))

# binary variable takes the value 1 if operation r of job j is processed by machine m else 0
X = {}
for j in N:
    for r in R:
        for m in m_dict[(j, r)]:
            X[(j, r, m)] = solver.BoolVar("")

# binary variable takes the value 1 if operation r of job j is processed immediately after the operation f of job i, both on machine m. else 0
Y = {}
for i in N_full:
    for f in R:
        for j in N_full:
            for r in R:
                for m in M:
                    Y[(i, f, j, r, m)] = solver.BoolVar("")
                    # Y[(i, f, j, r, m)] = solver.NumVar(0, 1, "")

# distribution
# deliver time of order (job) j
D = []
for j in N:
    D.append(solver.NumVar(0, solver.infinity(), ""))

# visiting time of customer (order) j by vehicle v
T = {}
for j in N_full:
    for v in V:
        T[(j, v)] = solver.NumVar(0, solver.infinity(), "")

# leaving time of vehicle v from production facility
S = []
for v in V:
    S.append(solver.NumVar(0, solver.infinity(), ""))

# visiting time of the last customer (order) in the tour of vehicle v
E = []
for v in V:
    E.append(solver.NumVar(0, solver.infinity(), ""))

# binary variable takes the value 1 if job j is delivered by vehicle else 0
Z = {}
for j in N:
    for v in V:
        Z[(j, v)] = solver.BoolVar("")

# binary variable takes the value 1 if job j is delivered after job i, by vehicle v else 0
U = {}
for i in N_full:
    for j in N_full:
        for v in V:
            U[(i, j, v)] = solver.BoolVar("")

# binary variable take the value 1 if vehicle v is used for delivery else 0
W = []
for v in V:
    W.append(solver.BoolVar(""))


## Constraint

# 3
for j in N:
    for r in R:
            solver.Add(solver.Sum([X[(j, r, m)] for m in m_dict[(j, r)]]) == 1, "ct3")

# 4
for j in N:
    for r in R:
        for m in m_dict[(j, r)]:
            solver.Add(X[(j, r, m)] <= a[(j, r, m)], "ct4")


# 5
for i in N:
    for f in R:
        for m in m_dict[(i, f)]:
            solver.Add(X[(i, f, m)] == solver.Sum(
                [Y[(i,f,j,r,m)] for j in N_plus for r in R]), "ct5")


# 6
for j in N:
    for r in R:
        for m in m_dict[(j, r)]:
            solver.Add(X[(j, r, m)] == solver.Sum(
        [Y[(i,f,j,r,m)] for i in N_minus for f in R]), "ct6")

# 7 Linear max
bc7 = {}
for i in N:
    for f in R:
        for m in m_dict[(i, f)]:
            bc7[(i, f, m)] = solver.NumVar(0, bigM, "")

z7 = {}
for j in N:
    for r in R_minus:
        for i in range(2):
            z7[(j, r, i)] = solver.BoolVar("")
            
for j in N:
    for r in R_minus:
        solver.Add(pi[(j, r)] >= gamma[(j,r-1)])
        # solver.Add(pi[(j, r)  ] <= gamma[(j,r-1)] + bigM*z7[(j, r, 0)])

        for i in N:
            for f in R:
                for m in m_dict[(i, f)]:
                    solver.Add(bc7[(i, f, m)] <= bigM * Y[(i, f, j, r, m)])
                    solver.Add(bc7[(i, f, m)] <= gamma[(i,f)] + bigM*(1-Y[(i, f, j, r, m)]))
                    solver.Add(bc7[(i, f, m)] >= gamma[(i,f)] - bigM*(1-Y[(i, f, j, r, m)]))
        solver.Add(pi[(j, r)] >= solver.Sum([bc7[(i, f, m)] for i in N for f in R for m in m_dict[(i, f)]]))

# 8
for j in N:
    for r in R:
        solver.Add(gamma[(j, r)] == pi[(j,r)] + solver.Sum([P[(j, r, m)] * X[(j,r,m)] for m in m_dict[(j, r)]]), "ct8")

# 9
for j in N:
    solver.Add(C[j-1] >= gamma[(j, R[-1])], "ct9")

# 10
# for m in M:
#     solver.Add(solver.Sum(
#         [Y[(0, f, j, r, m)] for j in N for f in R for r in R]) == 1, "ct10")

# 11
# for m in M:
#     solver.Add(solver.Sum(
#         [Y[(i, f, n+1, r, m)] for i in N for f in R for r in R]) == 1, "ct11")
    
# 12
for j in N:
    solver.Add(solver.Sum(
        [Z[(j, v)] for v in V]) == 1, "ct12")
    
# 13
for j in N:
    for v in V:
        solver.Add(Z[(j,v)] == solver.Sum(
            [U[((i, j, v))] for i in N_minus]), "ct13")

# 14 
for j in N:
    for v in V:
        solver.Add(solver.Sum(
            [U[(i,j,v)] for i in N_minus]) == solver.Sum(
                [U[(i,j,v)] for i in N_plus]), "ct14a")
        solver.Add(solver.Sum(
                [U[(i,j,v)] for i in N_plus]) <= 1, "ct14b")
        
# 15 
for v in V:
    solver.Add(solver.Sum(
        [U[(0,j,v)] for j in N]) == solver.Sum(
            [U[(i,n+1,v)] for i in N]), "ct15a")
    solver.Add(solver.Sum(
            [U[(i,n+1,v)] for i in N]) <= 1, "ct15b")
    
# 16
for v in V:
    solver.Add(solver.Sum(
        [Z[(j, v)]*theta[j-1] for j in N]) <= q[v-1], "ct16")
    
# 17 Add max ct
# Linearize binary * continuous
z17 = {}
for v in V:
    for j in N:
            z17[(v, j)] = solver.BoolVar("")
bc17 = {}
for j in N:
    for v in V:
        bc17[(j,v)] = solver.NumVar(0, bigM, "")

for v in V:
    solver.Add(T[(0, v)] == S[v-1], "ct15a")
    
    for j in N:
        solver.Add(bc17[(j,v)] <= bigM * Z[(j, v)])
        solver.Add(bc17[(j,v)] <= C[j-1] + bigM *(1 - Z[j, v]))
        solver.Add(bc17[(j,v)] >= C[j-1] - bigM *(1 - Z[j, v]))

        solver.Add(S[v-1] >= bc17[(j,v)])
        solver.Add(S[v-1] <= bc17[(j,v)] + bigM*z17[(v, j)])

    solver.Add(solver.Sum([z17[(v, j)] for j in N]) <= n - 1)
    
# 18 Linearize binary * continuous
bc18 = {}

for i in N:
    for j in N:
        for v in V:
            bc18[(i,j,v)] = solver.NumVar(0, bigM, "")

for j in N:
    for v in V:
        for i in N:
            solver.Add(bc18[(i,j,v)] <= bigM * U[(i,j,v)])
            solver.Add(bc18[(i,j,v)] <= T[(i,v)] + t[(i,j,v)] + bigM*(1-U[(i,j,v)]))
            solver.Add(bc18[(i,j,v)] >= T[(i,v)] + t[(i,j,v)] - bigM*(1-U[(i,j,v)]))
        solver.Add(T[(j,v)] == solver.Sum(
                [bc18[(i,j,v)] for i in N]), "ct18")

# 19 Linearize binary * continuous
bc19 = {} # dummy variable
for j in N:
    for v in V:
        bc19[(j,v)] = solver.NumVar(0, bigM, "")

for j in N:
    for v in V:
        solver.Add(bc19[(j,v)] <= bigM * Z[j,v])
        solver.Add(bc19[(j,v)] <= T[(j,v)] + bigM*(1-Z[j,v]))
        solver.Add(bc19[(j,v)] >= T[(j,v)] - bigM*(1-Z[j,v]))

    solver.Add(D[j-1] == solver.Sum(
        [bc19[(j,v)] for v in V]), "ct19")

# 20
for v in V:
    solver.Add(E[v-1] == S[v-1] + solver.Sum(
        [U[(i,j,v)] * t[(i,j,v)] for i in N_minus for j in N_plus])
        , "ct20")

# 21 Add Max constraint
z21 = {}
for v in V:
    for j in N:
            z21[(v, j)] = solver.BoolVar("")

for v in V:        
    for j in N:
        solver.Add(W[v-1] >= Z[(j, v)])
        solver.Add(W[v-1] <= Z[(j, v)] + bigM*z21[(v, j)])

    solver.Add(solver.Sum([z21[(v, j)] for j in N]) <= n - 1)

# 22 Max constraint for obj2

## Finish time
max_fin = [0] * len(N)
for j in N:
    max_fin[j-1] = solver.NumVar(0, bigM, "")

for j in N:
    solver.Add(max_fin[j-1] >= D[j-1] - tw[j-1, 1], "")


## Beginning time
max_beg = [0] * len(N)
for j in N:
    max_beg[j-1] = solver.NumVar(0, bigM, "")

for j in N:
    solver.Add(max_beg[j-1] >= tw[j-1, 0] - D[j-1], "")



# Objective
obj1 = solver.Sum(lamda[m-1] * P[(j, r, m)] * X[(j, r, m)] for j in N for r in R for m in m_dict[(j, r)])\
            + solver.Sum([F[v-1]*W[v-1] + Theta[v-1]*(E[v-1] - S[v-1]) for v in V])

obj2 = fei * solver.Sum([max_fin[j-1] for j in N])\
            + mu * solver.Sum([max_beg[j-1] for j in N])


{(1, 1): [1, 2], (1, 2): [1, 2], (1, 3): [1, 2], (2, 1): [1, 2], (2, 2): [1, 2], (2, 3): [1, 2], (3, 1): [1, 2], (3, 2): [1, 2], (3, 3): [1, 2]}


# IV. Flow control

## 1. First objective

In [148]:
### First objective
solver.Minimize(obj1)

# Solve
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

# Print solution.

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    obj1_best = solver.Objective().Value()

    for j in N:
        print(f"C{j} =", C[j-1].solution_value())
    
else:
    print("No solution found.")


# Bound the first obj
ct_obj1 = obj1 == obj1_best
ct_obj1 = solver.Add(ct_obj1)
solver.Minimize(obj2)

# Solve
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

# Print solution.
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    obj1_best = solver.Objective().Value()

    for j in N:
        print(f"C{j} = {C[j-1].solution_value()}")
    

else:
    print("No solution found.")

Solving with SCIP 8.1.0 [LP solver: Glop 9.9]
Total cost = 25170.0

C1 = 25.0
C2 = 21.0
C3 = 21.0
Solving with SCIP 8.1.0 [LP solver: Glop 9.9]
Total cost = 0.0

C1 = 25.0
C2 = 21.0
C3 = 25.0


## 2. Second objective

In [149]:
### Second objective

# Remove bound for obj1
ct_obj1.SetBounds(-solver.infinity(), solver.infinity())

# Set second obj function
solver.Minimize(obj2)

# Solve
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

# Print solution.

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    obj2_best = solver.Objective().Value()

    for j in N:
        print(f"C{j} =", C[j-1].solution_value())
    
else:
    print("No solution found.")


# Bound the first obj
ct_obj2 = obj2 == obj2_best
solver.Add(ct_obj2)
solver.Minimize(obj1)

# Solve
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

# Print solution.
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    obj1_best = solver.Objective().Value()

    for j in N:
        print(f"C{j} = {C[j-1].solution_value()}")
    

else:
    print("No solution found.")

Solving with SCIP 8.1.0 [LP solver: Glop 9.9]
Total cost = 0.0

C1 = 25.0
C2 = 21.0
C3 = 25.0
Solving with SCIP 8.1.0 [LP solver: Glop 9.9]
Total cost = 25170.0

C1 = 24.999999999999996
C2 = 21.0
C3 = 21.0
